In [1]:
## RAG implemetation

In [3]:
import re

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

SVC_ACC = !(gcloud config get-value core/account)
SVC_ACC = SVC_ACC[0]

PROJECT_NUMBER=str(re.search(r'\d+', SVC_ACC).group())

LOCATION="asia-southeast1"

FOLDER_NAME="."

In [4]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part

from crewai import Agent, Task, Crew, Process
from crewai_tools import tool
# from langchain_vertexai import ChatGemini
from crewai_tools.tools import FileReadTool
import os, requests, re, mdpdf, subprocess
from vertexai.preview.vision_models import ImageGenerationModel
from langchain_google_vertexai import ChatVertexAI
import uuid, os

# Initialize Gemini LLM
llm = ChatVertexAI(
    # model_name="gemini-pro", # Replace with your desired Gemini model
    # model = GenerativeModel("gemini-pro")
    # model = GenerativeModel("gemini-1.5-pro-preview-0215")
    model = "gemini-1.5-pro-002",
    project_id=os.getenv(PROJECT_ID), # Your Vertex AI project ID
    location="us-central1", # Your Vertex AI location
)


def generate_pro(input_prompt):
    model = GenerativeModel("gemini-pro")
    full_prompt = '''summarize the prompt below and do note prompt below will be send to imagen mode so please clean up any sensitve words and replace them into unblocked words like replace girl or lady can be replaced by female human and so on : ''' + input_prompt
    responses = model.generate_content(
    input_prompt,
    generation_config={
        "max_output_tokens": 8190,
        "temperature": 0.2,
        "top_p": 1
    },stream=False,)
    
    # print (responses.text)
    
    return(responses.text)



sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/jupyter/.config/sagemaker/config.yaml


In [5]:
# !pip install langchain-community langchain-core
%pip install faiss-cpu
%pip install -qU duckduckgo-search langchain-community


### RESTART KERNEL 

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
# !pip install -q pypdf
# !pip install -q faiss-gpu
# !pip install faiss-cpu

from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/GCP-Machine-learning-Certification.pdf")
pages = loader.load_and_split()


In [7]:
pages[0]



Document(metadata={'source': 'data/GCP-Machine-learning-Certification.pdf', 'page': 0}, page_content='\ue914Topic Cheatsheet for GCP’s Professional Machine Learning Engineer Beta Exam\nAuthors/contributors: David Chen, PhD\nCredits & disclaimers can be found in READMEsection of the source repository. Some references are included as %comments orhyperlinks the source file.\nAbbreviations\nCommon abbreviations . ML, machine learning; DL, deep learn-\ning; AI, artificial intelligence, CV, computer vision; GC(P), Google\nCloud (Platform); CI/CD: continuous integration / continuous de-\nlivery; SDK, software development kit; API, application program-\nming interface; K8s, Kubernetes; GKE, Google Kubernetes Engine.\nMLE, maximum likelihood estimation; ROC, receiver-operation\ncurve; AU(RO)C, area under the (receiver-operation) curve\nI. Preparation for ML\nUnderstanding the ”Data Science Steps for ML”\n1.Data extraction\n2.Exploratory data analysis\n3.Data preparation for the ML Task\n4.Model

In [8]:
%pip install -qU langchain-google-vertexai

# Instantiation
# Now we can instantiate our model object and generate embeddings:

# Check the list of Supported Models

from langchain_google_vertexai import VertexAIEmbeddings

# Initialize the a specific Embeddings Model version
embeddings = VertexAIEmbeddings(model_name="text-embedding-004")



Note: you may need to restart the kernel to use updated packages.


In [9]:
!pip install langchain_chroma

In [ ]:
#### Debug chroma

In [10]:
# from langchain_chroma import Chroma

# vector_store = Chroma(
#     collection_name="example_collection",
#     embedding_function=embeddings,
#     persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
# )


#### Debug duckduckgo

In [ ]:
# from langchain_community.tools import DuckDuckGoSearchRun

# search = DuckDuckGoSearchRun()

# search.invoke("Obama's first name?")

In [ ]:
from crewai import Agent, Task, Crew, Process
# from langchain_openai import ChatOpenAI
from langchain_core.retrievers import BaseRetriever
# from langchain_openai import OpenAIEmbeddings
# from langchain.embeddings import VertexAIEmbeddings
from langchain_google_vertexai import VertexAIEmbeddings


from langchain.tools import tool
from langchain_community.document_loaders import WebBaseLoader
import requests, os
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.vectorstores import Chroma
from langchain_community.tools import DuckDuckGoSearchRun

from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)


embedding_function = VertexAIEmbeddings(model_name="text-embedding-004")
# llm = ChatOpenAI(model="gpt-4-turbo-preview")

# Tool 1 : Save the news articles in a database
class SearchNewsDB:
    @tool("News DB Tool")
    def news(query: str):
        """Fetch news articles and process their contents."""
        API_KEY = os.getenv('65b726cdebac48f1ab5395c9802f6d48')  # Fetch API key from environment variable 65b726cdebac48f1ab5395c9802f6d48
        base_url = "https://newsapi.org/v2/everything"
        
        params = {
            'q': query,
            'sortBy': 'publishedAt',
            'apiKey': API_KEY,
            'language': 'en',
            'pageSize': 5,
        }
        
        response = requests.get(base_url, params=params)
        if response.status_code != 200:
            return "Failed to retrieve news."
        
        articles = response.json().get('articles', [])
        all_splits = []
        for article in articles:
            # Assuming WebBaseLoader can handle a list of URLs
            loader = WebBaseLoader(article['url'])
            docs = loader.load()

            text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
            splits = text_splitter.split_documents(docs)
            all_splits.extend(splits)  # Accumulate splits from all articles

        # Index the accumulated content splits if there are any
        if all_splits:
            vectorstore = Chroma.from_documents(all_splits, embedding=embedding_function, persist_directory="./chroma_db")
            retriever = vectorstore.similarity_search(query)
            return retriever
        else:
            return "No content available for processing."

# Tool 2 : Get the news articles from the database
class GetNews:
    @tool("Get News Tool")
    def news(query: str) -> str:
        """Search Chroma DB for relevant news information based on a query."""
        vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embedding_function)
        retriever = vectorstore.similarity_search(query)
        return retriever

# Tool 3 : Search for news articles on the web
search_tool = DuckDuckGoSearchRun()

# 2. Creating Agents
news_search_agent = Agent(
    role='News Seacher',
    goal='Generate key points for each news article from the latest news',
    backstory='Expert in analysing and generating key points from news content for quick updates.',
    tools=[SearchNewsDB().news],
    allow_delegation=True,
    verbose=True,
    llm=llm
)

writer_agent = Agent(
    role='Writer',
    goal='Identify all the topics received. Use the Get News Tool to verify the each topic to search. Use the Search tool for detailed exploration of each topic. Summarise the retrieved information in depth for every topic.',
    backstory='Expert in crafting engaging narratives from complex information.',
    tools=[GetNews().news, search_tool],
    allow_delegation=True,
    verbose=True,
    llm=llm
)

# 3. Creating Tasks
news_search_task = Task(
    description='Search for AI 2024 and create key points for each news.',
    agent=news_search_agent,
    tools=[SearchNewsDB().news],
    expected_output='A structured email accourding to the instructions'
)

writer_task = Task(
    description="""
    Go step by step.
    Step 1: Identify all the topics received.
    Step 2: Use the Get News Tool to verify the each topic by going through one by one.
    Step 3: Use the Search tool to search for information on each topic one by one. 
    Step 4: Go through every topic and write an in-depth summary of the information retrieved.
    Don't skip any topic.
    """,
    agent=writer_agent,
    context=[news_search_task],
    tools=[GetNews().news, search_tool],
    expected_output='A structured email accourding to the instructions'
)

# 4. Creating Crew
news_crew = Crew(
    agents=[news_search_agent, writer_agent],
    tasks=[news_search_task, writer_task],
    process=Process.sequential, 
    manager_llm=llm,
    expected_output='A structured email accourding to the instructions'
)

# Execute the crew to see RAG in action
result = news_crew.kickoff()
print(result)

2024-10-22 06:32:25,093 - 140433518380224 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: News Seacher
## Task: Search for AI 2024 and create key points for each news.


# Agent: News Seacher
## Thought: Thought: I need to search for news about AI 2024 using the News DB Tool and extract key points for each article.
## Using tool: News DB Tool
## Tool Input: 
"{\"query\": \"AI 2024\"}"
## Tool Output: 
Failed to retrieve news.


# Agent: News Seacher
## Thought: Thought: The News DB Tool failed.  I'll try a broader search term to see if I can get any results.  If that fails, I'll have to report that I couldn't retrieve any news.
## Using tool: News DB Tool
## Tool Input: 
"{\"query\": \"Artificial Intelligence 2024\"}"
## Tool Output: 
Failed to retrieve news.


# Agent: News Seacher
## Final Answer: 
I am unable to provide key points for AI 2024 news because the available tool, News DB Tool, failed to retrieve any relevant articles.  I attempted searches for both "AI 2024" and "Artificial Intelligence 2024" and both returned failures.


# Agent: Writer
## Task: 
  